# Teste: Consumo de Dados da Api do Spotify para analises sobre artistas e músicas.

## Objetivo

Avaliar conhecimentos nas linguagens Python e SQL e na engine de processamento Apache Spark.

## Descrição

Dado um conjunto de dados de músicas e artistas disponíveis em uma API pública, implemente um pipeline de processamento de dados com PySpark que realiza as seguintes operações:

* Consuma a API e extraia as informações de músicas e artistas dos seguintes genêros("Rock Nacional", "Piseiro/Arrocha" e "Pop Internacional";

* Armazene os dados em formato parquet, particionando por artista;

* Crie um dataframe com o endpoint "Get Artist"
* Crie um dataframe com o endpoint "Get Artist's Albums" e traga as músicas dos álbuns que estão no endpoint "Get Album Tracks".
* Crie um dataframe com o endpoint "Get Current User's Playlists"

* Crie uma tabela temporária em PySpark a partir do DataFrame de músicas dos artistas;
* Crie uma tabela temporária em Pyspark a partir do Dataframe de Playlists

* Execute uma consulta SQL que retorna os artistas do endpoint "Get Artist" que estão nas Playlists, ordenados por ordem alfabética.
* Crie um Dataframe com o resultado da consulta e salve em parquet

.

-------

Documentação da Api para consulta:
```
https://developer.spotify.com/documentation/web-api/
```

Para criar um Access Token na Spotify API, você precisa seguir os seguintes passos:

Acesse o Dashboard da API Spotify:
```
https://developer.spotify.com/dashboard/login
```
1. Faça login com sua conta do Spotify ou crie uma nova conta, caso ainda não tenha.
2. Crie um novo aplicativo clicando no botão "Create an App" e preencha as informações necessárias.
3. Após criar o aplicativo, você será direcionado para a página do aplicativo, onde poderá encontrar sua Client ID e Client Secret. Anote essas informações, pois elas serão necessárias para a autenticação.
4. Para gerar o Access Token, você precisará fazer uma solicitação GET/POST para o endpoint de autorização da API, passando sua Client ID e Client Secret como parâmetros. Você pode usar ferramentas como o Postman para realizar essa solicitação.
5. O endpoint de autorização irá retornar um Access Token que você pode utilizar para fazer requisições à API Spotify.

------------


## Atenção

- Leia a documentação dos endpoints para a extração dos dados, em alguns podem possuir número máximo de itens a serem retornados.

### Para realizar esta tarefa, os candidatos devem ter conhecimento em:

- PySpark (Spark SQL, Spark DataFrames);
- Consumo de API REST;
- armazenamento de dados;
- Transformações e consultas SQL em PySpark.
-------------
## Entregando o desafio

Faça uma cópia do desafio antes de começar a fazer o desafio e depois exporte para enviar!

Concluindo todos os passos informados, basta salvar o arquivo .ipynb do notebook e zipar juntamente com os parquet das tabelas, postar no seu github e enviar o link para ricardo.suhete@autoglass.com.br

A entrega da tarefa termina 7 dias após o recebimento deste notebook. Após o prazo será entendindo que o condidato desistiu da vaga.



-----------------------------------------------
### Caso não consiga rodar o spark no colab execute os comandos abaixo

In [ ]:
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
# !tar xf spark-3.3.1-bin-hadoop3.tgz

In [ ]:
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

---------------------------------------------------

In [1]:
!pip install -q findspark
!pip install -q unidecode
!pip install -q pyspark
import findspark
findspark.init()

import requests
import json
import unidecode
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Teste_Spark_Autoglass_candidatos") \
      .getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# Gerar token

import base64

client_id = '57233e49bf8f4a6f8eda75607d40e4e9'
client_secret = '2ea9be019937432c8e10697eb5bac8a1'
redirect_uri= 'http://localhost:8080/'


auth_str = f"{client_id}:{client_secret}"
base64_auth_str = base64.b64encode(auth_str.encode('utf-8')).decode('utf-8')


auth_url = 'https://accounts.spotify.com/api/token'
auth_headers = {
    'Authorization': 'Basic ' + base64_auth_str
}
auth_data = {
    'grant_type': 'client_credentials'
}

response = requests.post(auth_url, headers=auth_headers, data=auth_data)

if response.status_code == 200:
    body = response.json()
    token = body.get('access_token', None)
    if token:
        print(f'Token: {token}')
    else:
        print('Erro ao gerar.')
else:
    print('Erro número', response.status_code)

Token: BQABQZ5KsJ1vJmY-MVBFM0KGU-l44AEuGus7cNGwQZX9bAzkgsVIrNHvkmueZ_PRiH1LpJ3DsQy0MFeWe1XoWdwYo0YWoSbdka4y70JXvKIvtip8Wqg


In [3]:
# 1) Informações de artistas do genêros "Rock Nacional" e "Piseiro/Arrocha" (100 de cada):

url = 'https://api.spotify.com/v1/search'

access_token = token
headers = {
    'Authorization': f'Bearer {access_token}'
}

genres = ['rock brasileiro', 'piseiro', 'arrocha']

artistas = []

for genre in genres:
    genre_artists = []
    total_results = 100
    offset = 0
    while len(genre_artists) < total_results:
        params = {
            'q': f'genre:"{genre}"',
            'type': 'artist',
            'market': 'BR',
            'limit': 50,
            'offset': offset
        }
        response = requests.get(url, params=params, headers=headers)
        if response.status_code == 200:
            data = response.json()
            artists = data.get('artists', {}).get('items', [])
            if artists:
                genre_artists.extend(artists)
                next_page = data.get('artists', {}).get('next')
                if next_page:
                    offset += 50
                else:
                    break
            else:
                print(f"Nenhum artista de gênero '{genre}' encontrado.")
                break
        else:
            print(f"Erro na solicitação para o gênero '{genre}':", response.status_code)
            break
    artistas.extend(genre_artists)

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType
from pyspark.sql.functions import col

schema_artistas = StructType([
    StructField("name", StringType(), True),
    StructField("href", StringType(), True),
    StructField("id", StringType(), True),
    StructField("popularity", IntegerType(), True),
    StructField("type", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("external_urls", StructType([
        StructField("spotify", StringType(), True)
    ]), True),
    StructField("followers", StructType([
        StructField("href", StringType(), True),
        StructField("total", IntegerType(), True)
    ]), True),
    StructField("images", ArrayType(StructType([
        StructField("url", StringType(), True),
        StructField("height", IntegerType(), True),
        StructField("width", IntegerType(), True)
    ])), True),
    StructField("genres", ArrayType(StringType()), True)
])

df_artistas = spark.createDataFrame(artistas, schema_artistas)

# Para o gênero "Pop Internacional" foram aplicados filtros para excluir artistas brasileiros que também estavam classificados como "pop"

!pip install langdetect
from langdetect import detect

url = 'https://api.spotify.com/v1/search'

params = {
    'q': 'genre:"pop"',
    'type': 'artist',
    'market': 'US',
    'limit': 50
}

access_token = token
headers = {
    'Authorization': f'Bearer {access_token}'
}

english_artists = []

while len(english_artists) < 100:
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        artists = data.get('artists', {}).get('items', [])

        if not artists:
            print("Nenhum artista de gênero 'pop' encontrado.")
            break

        for artist in artists:
            artist_name = artist.get('name', '')
            if detect(artist_name) == 'en' and len(english_artists) < 100:
                english_artists.append(artist)

        next_page = data.get('artists', {}).get('next')
        if next_page:
            params['offset'] = params.get('offset', 0) + 50
        else:
            break
    else:
        print("Erro na solicitação:", response.status_code)
        break

df_pop = spark.createDataFrame(english_artists, schema=schema_artistas)

df_artistas = df_artistas.union(df_pop)

df_artistas.show(truncate=False)

# Opcionalmente verificar itens duplicados para posterior remoção:

name_counts = df_artistas.groupBy("name").count()
duplicate_names = name_counts.filter(col("count") > 1)
duplicate_names.show()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=45f7a7d6bd996e7f1faa0fc74188c258e24466eec6da2b9bc61c06f442d55f93
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
+-----------------------+---------------------------------------------------------+----------------------+----------+------+-------------------------------------+--------------------------------------------------------+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------

In [4]:
# 2) Armazenar dados em Parquet particionados por artistas

df_artistas.write.partitionBy('name').parquet('dados_artistas.parquet')

In [5]:
# 3) Criar um dataframe no endpoint "Get Artist"

from pyspark.sql.functions import udf, struct


def get_artist_info(artist_id):
    url = f'https://api.spotify.com/v1/artists/{artist_id}'
    access_token = token
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro na solicitação para o artista {artist_id}: {response.status_code}")
        return None

get_artist_info_udf = udf(get_artist_info)

artist_ids = df_artistas.select("id").rdd.flatMap(lambda x: x).collect()

artist_info_list = [get_artist_info(artist_id) for artist_id in artist_ids]

df_infos_artistas = spark.createDataFrame(artist_info_list)

df_infos_artistas.show(truncate=False)

+-------------------------------------------------------------------+--------------------------------+-----------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------+----------+------+-------------------------------------+
|external_urls                                                      |followers                       |genres                                                                                                                 |href                                                     |id                    |images                                                                                                                                  

In [6]:
# 4) (a) Criar um dataframe com o endpoint "Get Artist's Albums" e (b) trazer as músicas dos álbuns que estão no endpoint "Get Album Tracks"

# (a) Achar albums dos artistas

def get_artist_album(artist_id):
    url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups=album&limit=50'
    access_token = token
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro na solicitação para o artista {artist_id}: {response.status_code}")
        return None

get_artist_info_udf = udf(get_artist_info)

artist_ids = df_artistas.select("id").rdd.flatMap(lambda x: x).collect()

artist_album = [get_artist_album(artist_id) for artist_id in artist_ids]

# Schema para albuns

external_urls_schema = StructType([
    StructField("spotify", StringType(), nullable=True)
])


images_schema = StructType([
    StructField("url", StringType(), nullable=True),
    StructField("height", IntegerType(), nullable=True),
    StructField("width", IntegerType(), nullable=True)
])


artists_schema = StructType([
    StructField("external_urls", external_urls_schema, nullable=True),
    StructField("href", StringType(), nullable=True),
    StructField("id", StringType(), nullable=True),
    StructField("name", StringType(), nullable=True),
    StructField("type", StringType(), nullable=True),
    StructField("uri", StringType(), nullable=True)
])


items_schema = StructType([
    StructField("album_type", StringType(), nullable=True),
    StructField("total_tracks", IntegerType(), nullable=True),
    StructField("available_markets", ArrayType(StringType()), nullable=True),
    StructField("external_urls", external_urls_schema, nullable=True),
    StructField("href", StringType(), nullable=True),
    StructField("id", StringType(), nullable=True),
    StructField("images", ArrayType(images_schema), nullable=True),
    StructField("name", StringType(), nullable=True),
    StructField("release_date", StringType(), nullable=True),
    StructField("release_date_precision", StringType(), nullable=True),
    StructField("type", StringType(), nullable=True),
    StructField("uri", StringType(), nullable=True),
    StructField("album_group", StringType(), nullable=True),
    StructField("artists", ArrayType(artists_schema), nullable=True)
])


main_schema = StructType([
    StructField("href", StringType(), nullable=True),
    StructField("limit", IntegerType(), nullable=True),
    StructField("next", StringType(), nullable=True),
    StructField("offset", IntegerType(), nullable=True),
    StructField("previous", StringType(), nullable=True),
    StructField("total", IntegerType(), nullable=True),
    StructField("items", ArrayType(items_schema), nullable=True)
])


df_artist_album = spark.createDataFrame(artist_album, main_schema)

from pyspark.sql.functions import explode

df_exploded_album = df_artist_album.select("*", explode("items").alias("exploded_items"))

df_albuns = df_exploded_album.select("exploded_items.*")

# (b) trazer as músicas dos álbuns

def get_track(album_id):
    url = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
    access_token = token
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()['items']
    else:
        print(f"Erro na solicitação para tracks {album_id}: {response.status_code}")
        return None

from pyspark.sql.types import MapType, StringType

get_album_tracks_udf = udf(get_track, ArrayType(MapType(StringType(),StringType())))

df_tracks = df_albuns.select(get_album_tracks_udf(col("id")).alias("tracks"))

df_exploded_track = df_tracks.select(explode("tracks").alias("tracks_info"))

df_musicas = df_exploded_track.select(explode("tracks_info").alias("tracks_info","faixas"))

# Df com músicas
df_musicas = df_musicas.filter(col("tracks_info") == "name")

In [7]:
# 5) Criar um dataframe com o endpoint "Get Current User's Playlists"

# É necessário criar um token com escopo 'playlist-read-private'

authorization_url = f'https://accounts.spotify.com/authorize?client_id={client_id}&response_type=code&redirect_uri={redirect_uri}&scope=playlist-read-private'

print("inserir código código de autorização:")
print(authorization_url)
authorization_code = input("Código de Autorização: ")

token_url = 'https://accounts.spotify.com/api/token'
token_data = {
    'grant_type': 'authorization_code',
    'code': authorization_code,
    'redirect_uri': redirect_uri,
    'client_id': client_id,
    'client_secret': client_secret,
}

response = requests.post(token_url, data=token_data)
response_data = response.json()

access_token = response_data['access_token']

# Requisição para 25 playlists
url = 'https://api.spotify.com/v1/me/playlists?limit=25'

headers = {
    'Authorization': f'Bearer {access_token}'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()

from pyspark.sql.types import BooleanType

schema_playlist = StructType([
    StructField("href", StringType(), True),
    StructField("limit", IntegerType(), True),
    StructField("next", StringType(), True),
    StructField("offset", IntegerType(), True),
    StructField("previous", StringType(), True),
    StructField("total", IntegerType(), True),
    StructField("items", ArrayType(
        StructType([
            StructField("collaborative", BooleanType(), True),
            StructField("description", StringType(), True),
            StructField("external_urls", MapType(StringType(), StringType()), True),
            StructField("href", StringType(), True),
            StructField("id", StringType(), True),
            StructField("images", ArrayType(
                StructType([
                    StructField("url", StringType(), True),
                    StructField("height", StringType(), True),
                    StructField("width", StringType(), True),
                ])
            ), True),
            StructField("name", StringType(), True),
            StructField("owner", StructType([
                StructField("external_urls", MapType(StringType(), StringType()), True),
                StructField("href", StringType(), True),
                StructField("id", StringType(), True),
                StructField("type", StringType(), True),
                StructField("uri", StringType(), True),
                StructField("display_name", StringType(), True),
            ]), True),
            StructField("public", BooleanType(), True),
            StructField("snapshot_id", StringType(), True),
            StructField("tracks", StructType([
                StructField("href", StringType(), True),
                StructField("total", IntegerType(), True),
            ]), True),
            StructField("type", StringType(), True),
            StructField("uri", StringType(), True),
            StructField("primary_color", StringType(), True),
        ])
    ), True),
])

df_playlists = spark.createDataFrame([data], schema=schema_playlist)

item_schema = StructType([
    StructField("collaborative", BooleanType(), True),
    StructField("description", StringType(), True),
    StructField("external_urls", MapType(StringType(), StringType()), True),
    StructField("href", StringType(), True),
    StructField("id", StringType(), True),
    StructField("images", ArrayType(
        StructType([
            StructField("url", StringType(), True),
            StructField("height", StringType(), True),
            StructField("width", StringType(), True),
        ])
    ), True),
    StructField("name", StringType(), True),
    StructField("owner", StructType([
        StructField("external_urls", MapType(StringType(), StringType()), True),
        StructField("href", StringType(), True),
        StructField("id", StringType(), True),
        StructField("type", StringType(), True),
        StructField("uri", StringType(), True),
        StructField("display_name", StringType(), True),
    ]), True),
    StructField("public", BooleanType(), True),
    StructField("snapshot_id", StringType(), True),
    StructField("tracks", StructType([
        StructField("href", StringType(), True),
        StructField("total", IntegerType(), True),
    ]), True),
    StructField("type", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("primary_color", StringType(), True),
])

playlist_schema = StructType([
    StructField("href", StringType(), True),
    StructField("limit", IntegerType(), True),
    StructField("next", StringType(), True),
    StructField("offset", IntegerType(), True),
    StructField("previous", StringType(), True),
    StructField("total", IntegerType(), True),
    StructField("items", ArrayType(item_schema), True),
])

df_playlists = df_playlists.select("href", "limit", "next", "offset", "previous", "total", explode("items").alias("item"))

item_schema = StructType([
    StructField("collaborative", BooleanType(), True),
    StructField("description", StringType(), True),
    StructField("external_urls", MapType(StringType(), StringType()), True),
    StructField("href", StringType(), True),
    StructField("id", StringType(), True),
    StructField("images", ArrayType(
        StructType([
            StructField("url", StringType(), True),
            StructField("height", StringType(), True),
            StructField("width", StringType(), True),
        ])
    ), True),
    StructField("name", StringType(), True),
    StructField("owner", StructType([
        StructField("external_urls", MapType(StringType(), StringType()), True),
        StructField("href", StringType(), True),
        StructField("id", StringType(), True),
        StructField("type", StringType(), True),
        StructField("uri", StringType(), True),
        StructField("display_name", StringType(), True),
    ]), True),
    StructField("public", BooleanType(), True),
    StructField("snapshot_id", StringType(), True),
    StructField("tracks", StructType([
        StructField("href", StringType(), True),
        StructField("total", IntegerType(), True),
    ]), True),
    StructField("type", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("primary_color", StringType(), True),
])


playlist_schema = StructType([
    StructField("href", StringType(), True),
    StructField("limit", IntegerType(), True),
    StructField("next", StringType(), True),
    StructField("offset", IntegerType(), True),
    StructField("previous", StringType(), True),
    StructField("total", IntegerType(), True),
    StructField("items", ArrayType(item_schema), True),
])


df_playlists = df_playlists.select("href", "limit", "next", "offset", "previous", "total", "item.*")

df_playlists.show(truncate=False)

inserir código código de autorização:
https://accounts.spotify.com/authorize?client_id=57233e49bf8f4a6f8eda75607d40e4e9&response_type=code&redirect_uri=http://localhost:8080/&scope=playlist-read-private
Código de Autorização: AQD3TGLuuIM-e5tU-iLTtY5Pd0zs_JmKYWk91_VuFAaEgJqUbCWUN4Tsdq5AJHRZSGlFxCre8j9NPdb5sOMYHio3wZ5MgMP62PzpvVQkP40uFDwjF8K8mm0DZ7UOXnKOIS6hkvHSK_pHN4HS07FqbWz-EmnCDWJ5D0pN2jN83jTQvQn0A5n8Jg03cJ6Rzjh7Qg
+--------------------------------------------------------------------------------------+-----+----+------+--------+-----+-------------+-------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------+-----------------------------------------------------------+----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
# 6) Criar uma tabela temporária em PySpark a partir do DataFrame de músicas dos artistas

df_musicas.createOrReplaceTempView("musicas")

In [11]:
# 7) Criar uma tabela temporária em Pyspark a partir do Dataframe de Playlists

df_playlists.createOrReplaceTempView("playlists")

In [13]:
# 8) Executar uma consulta SQL que retorna os artistas do endpoint "Get Artist" que estão nas Playlists, ordenados por ordem alfabética.

# É necessário extrair os artistas que estão em minhas playlists
playlist_ids = df_playlists.select("id").rdd.flatMap(lambda x: x).collect()

playlist_data_list = []

for i in playlist_ids:
    url = f'https://api.spotify.com/v1/playlists/{i}/tracks?fields=items%28track%28artists%28name%29%29%29'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        playlist_data_list.append(data)
    else:
        error_code = response.status_code
        print(f'Erro {error_code} ao obter dados da playlist')

def extract_names(data):
    names = []

    def extract_recursively(item):
        if isinstance(item, dict):
            for key, value in item.items():
                if key == 'name':
                    names.append(value)
                else:
                    extract_recursively(value)
        elif isinstance(item, list):
            for element in item:
                extract_recursively(element)

    extract_recursively(data)
    return names

names = extract_names(playlist_data_list)

from pyspark.sql import Row

linhas = [Row(artistas=artista) for artista in names]

rdd = spark.sparkContext.parallelize(linhas)

df_consulta = spark.createDataFrame(rdd)

df_infos_artistas.createOrReplaceTempView("get_artist")
df_consulta.createOrReplaceTempView("playlists")

# Executar a consulta SQL
consulta_sql = """
    SELECT DISTINCT a.name
    FROM get_artist a
    INNER JOIN playlists e ON a.name = e.artistas
    ORDER BY a.name
"""

resultado = spark.sql(consulta_sql)

resultado.show()

+--------------------+
|                name|
+--------------------+
|      Barão Vermelho|
|     Biquini Cavadão|
|              CPM 22|
|     Cachorro Grande|
|     Capital Inicial|
|          Charli XCX|
|   Charlie Brown Jr.|
|       Chico Science|
|           Dead Fish|
|Detonautas Roque ...|
|         Don Toliver|
|           Fetty Wap|
|Florence + The Ma...|
|              Fresno|
|              Hateen|
|                Ira!|
|          Jota Quest|
|        Joyner Lucas|
|             Juicy J|
|       Legião Urbana|
+--------------------+
only showing top 20 rows



In [14]:
# 9) Criar um Dataframe com o resultado da consulta e salvar em parquet

resultado.write.mode("overwrite").parquet("resultados.parquet")